# <center> Homework 7: Support Vector Machines </center>
<center> Anushna Prakash </center>
<center> May 28, 2021 </center>

$$ \min_{\alpha \in \mathbb{R}^n} F(\alpha):= \frac{1}{n} \sum_{i=1}^{n} \ell(y_i, (K\alpha)_i) + \lambda \alpha^T K \alpha$$  
$$ \min_{\alpha \in \mathbb{R}^n} F(\alpha):= \frac{1}{n} \sum_{i=1}^{n} (\max \{0, 1 - y_i (K\alpha)_i\})^2 + \lambda \alpha^T K \alpha$$  
$$ \nabla F(\alpha) = -\frac{2}{n} \sum_{i=1}^{n} K_i y_i  \max \{0, 1-y_i K_i^T \alpha\} + 2\lambda K \alpha $$  
$$
    \nabla F(\alpha) = 
    \begin{cases}
        -\frac{2}{n} \sum_{i=1}^{n} K_i y_i (1 - y_i K_i^T \alpha) + 2 \lambda K \alpha, & \text{for } 1-y_i(K\alpha)_i > 0\\
        2 \lambda K \alpha, & \text{for } 1-y_i(K\alpha)_i <= 0\\
    \end{cases}
$$

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
def kernal(x, y, b, p):
    return (np.dot(x.T, y) + b)**p

In [3]:
def computegram(k, x, b, p):
    n = x.shape[0]
    K = np.empty(shape = (n, n))
    
    for i, v in enumerate(x):
        for j, w in enumerate(x):
            K[i][j] = k(v, w, b, p)
    
    return K

In [4]:
def kernaleval(k, x, x_star, b, p):
    kernal_evals = np.empty(len(x))
    for i, v in enumerate(x):
        kernal_evals[i] = k(v, x_star, b, p)
    
    return kernal_evals

In [5]:
def svm_f(alpha, X, y, lambda_, b, p):
    n = X.shape[0]
    K = computegram(kernal, X, b, p) # nxn matrix
    K_a = np.dot(K, alpha)
    loss = 0
    for i in range(n):
        cond = 1.0 - y[i] * K_a[i]
        if cond > 0:
            loss += cond**2
#     loss = np.maximum(0, 1.0 - np.multiply(y, np.dot(K, alpha)))

    return (1.0/n) * loss + lambda_ * (alpha.T @ K @ alpha)

In [6]:
def svm_grad(alpha, X, y, lambda_, b, p):
    n = len(X)
    K = computegram(kernal, X, b, p)
    K_a = np.dot(K, alpha)
#     loss = 1.0 - np.multiply(y, np.dot(K, alpha))
    reg = 2.0 * lambda_ * K_a
    loss_grad = np.zeros(reg.shape)
    
    for i in range(n):
        cond = 1.0 - y[i] * K_a[i]
        K_y = K[i] * y[i]
        if cond > 0:
            loss_grad += -2 * K_y * cond
        
    return (1/n) * loss_grad + reg 

In [7]:
def backtracking(eta_init, decay_rate, prop_constant, alpha, *args):
    eta = eta_init
    
    def decrease_condition(eta):
        grad = svm_grad(alpha, *args)
        left = svm_f(alpha - eta * grad, *args) - svm_f(alpha, *args)
        right = eta * prop_constant * np.linalg.norm(grad)**2
        return left <= right
    
    while not decrease_condition(eta):
        eta *= decay_rate
    
    return eta

In [8]:
def mysvm(eta_init, tol, alpha_init, *args):
    theta = alpha_init.copy()
    alpha = alpha_init.copy()
    iters = [alpha]
    grad = svm_grad(alpha, *args)
    decay_rate = 0.8
    prop_constant = 0.3
    t = 0
    
    while np.linalg.norm(grad) > tol:
        print(grad)
        eta = backtracking(eta_init, decay_rate, prop_constant, theta, *args)
        alpha_t = theta - eta * svm_grad(theta, *args)
        theta = alpha_t + (t/(t + 3)) * (alpha_t - alpha)
        grad = svm_grad(alpha_t, *args)
        iters.append(alpha_t)
        alpha = alpha_t
        t += 1
    
    return iters

In [9]:
# Download the data
digits = load_digits().data
y = load_digits().target

# Normalize and split 80/20 training and test
digits = normalize(digits, norm = 'l2', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(digits, y, train_size = 0.8, random_state = 12345)

In [10]:
digits = np.unique(y)
classifiers = {}

In [11]:
y_train_0 = np.where(y_train == 0, 1, -1)
eta_init = 0.000001
tol = 1.0
alpha_init = np.zeros(X_train.shape[0])
lam = 10.0
b = 1
p = 7
svm_d = mysvm(eta_init, tol, alpha_init, X_train, y_train_0, lam, b, p)

[49.32929849 73.19343087 57.71184788 ... 68.70622177 69.47171828
 67.93753343]


KeyboardInterrupt: 

In [ ]:
# for d in digits:
#     # train a classifier on y_i^d = 1 if y_i = d, -1 else
#     y_train_d = np.where(y_train == d, 1, -1)
#     eta_init = 0.5
#     tol = 1.0
#     alpha_init = np.zeros(X_train.shape[0])
#     lam = 10.0
#     b = 1
#     p = 7
#     svm_d = mysvm(eta_init, tol, alpha_init, X_train, y_train_d, lam, b, p)
#     classifiers[d] = svm_d